In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


df_test =   pd.read_csv(
    "full_reasoning_split/mmlu_pro_stem_reasoning_score.tsv",
    sep="\t",
    header=0, 
    escapechar="\\",
)


#df = df[(df['category'] == 'math') | (df['category'] == 'biology')]
df_test = df_test.drop_duplicates(subset=['question'])
df_test['masj_num_reasoning_steps'].value_counts()

masj_num_reasoning_steps
medium    7263
low       2706
high      1031
Name: count, dtype: int64

In [6]:
df_test['key'] = df_test['question'] + df_test['options']

In [8]:
cot = pd.read_csv('mmlu_deepseek_v3.tsv',sep='\t')
cot = cot[~cot['distill_response'].isna()]
cot = cot[cot['distill_ans_correct']==True]
cot = cot.drop_duplicates(subset=['question'])
cot['key'] = cot['question'] + cot['options']
cot['masj_num_reasoning_steps'] = cot['key'].map(df_test.set_index('key')['masj_num_reasoning_steps'])
cot


,src,answer,options,category,question,cot_content,question_id,answer_index,total_tokens,meta_cluster,base_cluster,distill_ans_correct,distill_response,distill_answer,key,masj_num_reasoning_steps
0,ori_mmlu-clinical_knowledge,E,"['Unilateral oval shaped pupil.', 'Bilateral o...",health,Which one of the following represents a IIIrd ...,NaN,6777,4,98,Miscellaneous,Viruses and Diseases,True,"To answer this question, let's carefully analy...",5.0,Which one of the following represents a IIIrd ...,medium
2,ori_mmlu-conceptual_physics,J,"['hot to cold without external energy', 'cold ...",physics,The second law of thermodynamics tells us that...,NaN,10058,9,89,Biology Questions,Chemistry & Biology Reactions,True,"Alright, let's tackle this question step by st...",10.0,The second law of thermodynamics tells us that...,medium
3,ori_mmlu-electrical_engineering,H,"['15 µF', '10 µF', '5 µF', '0.1 µF', '2 µF', '...",engineering,Five capacitors each of 5 µF are connected in ...,NaN,12128,7,67,Miscellaneous,Electrical Engineering Questions,True,To find the equivalent capacitance of five cap...,8.0,Five capacitors each of 5 µF are connected in ...,medium
4,stemez-Optics,H,"['4 cm, (1 / 2) cm, 1 degree', '4.5 cm, (3 / 4...",physics,"In a simple telescope, it is given that F, the...",NaN,10191,7,318,Scientific Calculations,Wavelength & Energy Calculations,True,"Alright, let's tackle this telescope problem s...",8.0,"In a simple telescope, it is given that F, the...",high
5,ori_mmlu-business_ethics,A,"['Caveat emptor, Buyer beware, Regulation, Nat...",business,Traditionally consumer rights were based on th...,NaN,496,0,188,Economics & Finance MCQs,Business & Marketing Queries,True,Let's analyze the question and the options ste...,1.0,Traditionally consumer rights were based on th...,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,stemez-Business,E,"['$7200, $5243, $1957', '$6500, $5243, $842', ...",business,"Ted Logan, as a result of a recent illness, sp...",NaN,560,4,339,Economics & Finance MCQs,Business Finance Questions,True,To determine Mr. Logan's total medical expense...,5.0,"Ted Logan, as a result of a recent illness, sp...",high
3326,ori_mmlu-high_school_mathematics,I,"['$f(4)$', '$f(f^{-1}(2))$', '1', '$f^{-1}(4)$...",math,"Suppose $f(x)$ is an invertible function, and ...",NaN,7751,8,81,Mathematical Problem Solving,Elementary Math Calculations,True,11. $-1$\n\nTo determine the value of \( f(f(2...,9.0,"Suppose $f(x)$ is an invertible function, and ...",medium
3327,theoremQA-Physics,J,"['1.8', '1.5', '0.5', '1.4', '0.7', '1.0', '2....",physics,The shock absorbers in an old car with mass 10...,NaN,10339,9,134,Scientific Calculations,Physics Calculation Questions,True,"To solve this problem, we need to determine th...",10.0,The shock absorbers in an old car with mass 10...,medium
3328,stemez-Physics,G,"['0.6 ampere', '2 amperes', '0.2 ampere', '1 a...",physics,Find the current through the filament of a lig...,NaN,9782,6,82,Engineering Calculations,Electric Circuit Calculations,True,To find the current through the filament of th...,7.0,Find the current through the filament of a lig...,low


In [9]:
cot['masj_num_reasoning_steps'].value_counts()

masj_num_reasoning_steps
medium    1623
low        631
high       206
Name: count, dtype: int64

In [16]:

train_valid_df, test_df = train_test_split(cot, test_size=0.2, random_state=42)
test_df.to_csv("cot_reasoning_split/test_combined_reasoning.tsv", sep="\t", index=False)
print(f"Тестовый датасет: {len(test_df)} примеров сохранён в 'test.tsv'.")

train_valid_df = train_valid_df.sort_values(by="masj_num_reasoning_steps", ascending=False)
N = len(train_valid_df)
print(f"Всего обучающих+валидационных примеров: {N}")

def split_and_save_data(
    df,
    complexity_col='masj_num_reasoning_steps',
    thresholds=[(None, 'low', 'easy'), ('low', 'high', 'middle'), ('high', None, 'hard')],
    test_size=0.1,
    random_state=42,
    output_prefix=""
    ):
    """
    Разделяет данные по уровням сложности и сохраняет в TSV-файлы.
    """
    for lower, upper, suffix in thresholds:
        if lower is None:
            filtered = df[df[complexity_col] ==  upper].sample(150,random_state=42)
        elif upper is None:
            filtered = df[df[complexity_col] == lower].sample(150,random_state=42)
        else:
            filtered = df[(df[complexity_col] != lower) & (df[complexity_col] != upper)].sample(150,random_state=42)
        
        filtered = filtered.reset_index(drop=True)
        
        train, valid = train_test_split(filtered, test_size=test_size, random_state=random_state)
        
        train.to_csv(f"cot_reasoning_split/{output_prefix}train_df_{suffix}.tsv", sep='\t', index=False)
        valid.to_csv(f"cot_reasoning_split/{output_prefix}valid_df_{suffix}.tsv", sep='\t', index=False)

split_and_save_data(train_valid_df, complexity_col='masj_num_reasoning_steps')

Тестовый датасет: 525 примеров сохранён в 'test.tsv'.
Всего обучающих+валидационных примеров: 2097


In [17]:
test = pd.read_csv("cot_reasoning_split/test_combined_reasoning.tsv", sep="\t").dropna(subset='masj_num_reasoning_steps')
class_count = test['masj_num_reasoning_steps'].value_counts()
min_count =class_count.min()
balanced_dfs = []
for classe in test['masj_num_reasoning_steps'].dropna().unique():
    class_df = test[test['masj_num_reasoning_steps'] == classe]
    sampled_class_df = class_df.sample(n=min_count,random_state=42)
    balanced_dfs.append(sampled_class_df)
    
balanced_df = pd.concat(balanced_dfs)
balanced_df.to_csv("cot_reasoning_split/test_balanced_combined_reasoning.tsv", sep="\t", index=False)
    

In [19]:
random_train_valid_df = train_valid_df.sample(150,random_state=42)
train, valid = train_test_split(random_train_valid_df, test_size=0.1, random_state=42)

In [20]:
train.to_csv(f"cot_reasoning_split/train_df_random.tsv", sep='\t', index=False)
valid.to_csv(f"cot_reasoning_split/valid_df_random.tsv", sep='\t', index=False)